# Generate the map

Imports

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import os

Setup paths

In [ ]:
data_dir = "data"
output_dir = "output"
base_dir = ".."  # because we need to climb out of sub directory
input_data_filename = "umd_buildings.shp"
output_map_filename = "umd_buildings.png"
path_to_input_data = os.path.join(base_dir, data_dir, input_data_filename)
path_to_output_map = os.path.join(base_dir, output_dir, output_map_filename)

Read spatial data into geopandas

In [ ]:
geodata = gpd.read_file(path_to_input_data)

Plot data on map

In [ ]:
# Setup plot using shapefile
ax = geodata.plot(figsize=(6,6), alpha=0.5, edgecolor="k")
# Add basemap
ctx.add_basemap(ax, crs=geodata.crs.to_string(), source=ctx.providers.CartoDB.Voyager)
# Save map to file
# not using plt.show() because it seems to mess with saving the map in jupyter notebooks
# see https://stackoverflow.com/questions/9012487/matplotlib-pyplot-savefig-outputs-blank-image
plt.savefig(path_to_output_map)

# Generate the HTML

Imports

In [ ]:
from jinja2 import Environment, FileSystemLoader
from datetime import datetime

Setup paths

In [ ]:
input_template_filename = "map_test.html"

Load HTML template

In [ ]:
# setup environment
env = Environment(loader=FileSystemLoader(os.path.join(base_dir, data_dir)))
# load template from environment
loaded_template = env.get_template(input_template_filename)

Insert dynamic content into template

In [ ]:
content = {
    "title": "My Report",
    "map_title": "My Basic Map of UMD",
    # just need map filename since it's in the same directory as report
    "map": output_map_filename,
}

Export HTML to string

In [ ]:
html_string = loaded_template.render(content)

Save HTML to file

In [ ]:
output_html_filename = \
    f'report_with_map_{datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")}.html'
path_to_output_html = os.path.join(base_dir, output_dir, output_html_filename)
loaded_template.stream(content).dump(path_to_output_html)

# Generate the PDF

Imports

In [ ]:
from weasyprint import HTML

Setup paths

In [ ]:
# add time to outputs so they don't get overwritten
output_report_filename = \
    f'report_with_map_{datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")}.pdf'
path_to_output_report = os.path.join(base_dir, output_dir, output_report_filename)

Generate PDF

In [ ]:
base_url = os.path.realpath(os.path.join(base_dir, output_dir))
htmldoc = HTML(string=html_string, base_url=base_url)
htmldoc.write_pdf(path_to_output_report)

# References

[Creating PDF Reports with Pandas, Jinja and WeasyPrint](https://pbpython.com/pdf-reports.html)
[Python PDF Generation from HTML with WeasyPrint](https://dev.to/bowmanjd/python-pdf-generation-from-html-with-weasyprint-538h)